In [1]:
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import sys
from operator import add
from functools import reduce
import mplhep as hep
plt.style.use([hep.style.CMS])
import vector
vector.register_awkward()

In [2]:
# import all the file we need
# events = ak.from_parquet("/eos/user/z/zhenxuan/hhwwgg/merged_nominal.parquet")
events_nano = uproot.open("root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAODv7/TTGJets_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/260000/AB100F23-BA74-B742-A178-D460A44C72D4.root")

In [12]:
events_nano["Events"].keys()

['run',
 'luminosityBlock',
 'event',
 'HTXS_Higgs_pt',
 'HTXS_Higgs_y',
 'HTXS_stage1_1_cat_pTjet25GeV',
 'HTXS_stage1_1_cat_pTjet30GeV',
 'HTXS_stage1_1_fine_cat_pTjet25GeV',
 'HTXS_stage1_1_fine_cat_pTjet30GeV',
 'HTXS_stage1_2_cat_pTjet25GeV',
 'HTXS_stage1_2_cat_pTjet30GeV',
 'HTXS_stage1_2_fine_cat_pTjet25GeV',
 'HTXS_stage1_2_fine_cat_pTjet30GeV',
 'HTXS_stage_0',
 'HTXS_stage_1_pTjet25',
 'HTXS_stage_1_pTjet30',
 'HTXS_njets25',
 'HTXS_njets30',
 'btagWeight_CSVV2',
 'btagWeight_DeepCSVB',
 'CaloMET_phi',
 'CaloMET_pt',
 'CaloMET_sumEt',
 'ChsMET_phi',
 'ChsMET_pt',
 'ChsMET_sumEt',
 'nCorrT1METJet',
 'CorrT1METJet_area',
 'CorrT1METJet_eta',
 'CorrT1METJet_muonSubtrFactor',
 'CorrT1METJet_phi',
 'CorrT1METJet_rawPt',
 'nElectron',
 'Electron_deltaEtaSC',
 'Electron_dr03EcalRecHitSumEt',
 'Electron_dr03HcalDepth1TowerSumEt',
 'Electron_dr03TkSumPt',
 'Electron_dr03TkSumPtHEEP',
 'Electron_dxy',
 'Electron_dxyErr',
 'Electron_dz',
 'Electron_dzErr',
 'Electron_eCorr',
 'Electron

In [144]:
events_nano = uproot.open("root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAODv7/GluGluToRadionToHHTo2G4Q_M-300_narrow_TuneCP5_PSWeights_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/250000/84A472F4-0A01-8142-A979-BB10929C783D.root")
events_nano = events_nano["Events"].arrays(["Photon_pt", "Photon_eta", "Photon_phi", "Photon_mass",
        "Photon_pixelSeed", "Photon_mvaID", "Photon_electronVeto",
        "Photon_sieie",
        "Photon_r9", "Photon_hoe", "Photon_pfRelIso03_chg", "Photon_pfRelIso03_all",
        "Photon_isScEtaEB", "Photon_isScEtaEE","Photon_genPartFlav"],library='ak',how="zip")

In [147]:
events_nano.Photon.pt[events_nano.Photon.pt>200]

<Array [[], [], [], [], ... [], [], [], []] type='19792 * var * float32'>

In [54]:
# get the whole events number
import os 
p = os.popen(cmd).readlines()
events_number = []
for i in range(len(p)):
    events_number.append(int(p[i].split("Loaded")[-1].split("events")[0]))
sum(events_number)

AttributeError: 'list' object has no attribute 'close'

In [52]:
# get the whole events number
import os 
cmd = 'grep "load_events" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
p = os.popen(cmd).readlines()
events_number = []
for i in range(len(p)):
    events_number.append(int(p[i].split("Loaded")[-1].split("events")[0]))
sum(events_number)

288160

In [139]:
# get all events eff with combine all the log file together
import os 
import numpy as np
cmd_events = 'grep "load_events" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
cmd_HHWWtagger_events = 'grep "my_HHWW_Preselection : event set" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
cmd_photon = 'grep "Photon Selection" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
cmd_lepton = 'grep "Lepton Selection" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
cmd_jet = 'grep "OnlyFourJet_category" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
events = os.popen(cmd_events).readlines()
HHWWtagger_events = os.popen(cmd_HHWWtagger_events).readlines()
photon = os.popen(cmd_photon).readlines()
lepton = os.popen(cmd_lepton).readlines()
jet = os.popen(cmd_jet).readlines()
events_number = []
HHWWtagger_events_number = []
Photon_eff = []
Lepton_eff = []
Jet_eff = []
Jet_eff = []
for i in range(len(events)):
    events_number.append(int(events[i].split("Loaded")[-1].split("events")[0]))
    HHWWtagger_events_number.append(float(HHWWtagger_events[i].split('(')[0].split()[-1]))
    Photon_eff.append(float(HHWWtagger_events[i].split('(')[0].split()[-1]) * float(photon[i].split(':')[-2].split(' ')[1])/int(events[i].split("Loaded")[-1].split("events")[0]) )
    Lepton_eff.append(float(HHWWtagger_events[i].split('(')[0].split()[-1]) * float(lepton[i].split(':')[-2].split(' ')[1])/int(events[i].split("Loaded")[-1].split("events")[0]) )
    Jet_eff.append(float(jet[i].split(':')[-2].split(' ')[1]))
Photon_eff = float(np.mean(Photon_eff))
Lepton_eff = float(np.mean(Lepton_eff))
Jet_eff = float(np.mean(Jet_eff))
all_HHWWtagger_events = float(np.sum(HHWWtagger_events_number))
all_events = float(np.sum(events_number))
print("all events: %i" %(all_events) )
print("Photon numbers:%i" %(all_events*Photon_eff), "Photon Selection eff: %f" %(Photon_eff) )
print("Lepton numbers:%i" %(all_events*Lepton_eff), "Lepton Selection eff: %f" %(Lepton_eff) )
print("Jet numbers:%i" %(all_HHWWtagger_events*Jet_eff), "Jet Selection eff: %f" %(all_HHWWtagger_events*Jet_eff / all_events) )


all events: 288160
Photon numbers:108592 Photon Selection eff: 0.376850
Lepton numbers:108489 Lepton Selection eff: 0.376492
Jet numbers:42982 Jet Selection eff: 0.149163


In [3]:
cmd = 'dasgoclient --query="file dataset=/QCD_HT500to700_TuneCP5_13TeV-madgraph-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/NANOAODSIM"'
root_files = os.popen(cmd).readlines()
print(root_files)

KeyboardInterrupt: 

In [2]:
for i in range(root_files):
    root_files[0]

'/store/mc/RunIIFall17NanoAODv7/QCD_HT500to700_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/60000/169BC0BF-5437-4C4A-A10A-A9FD01852E26.root\n'

In [129]:
HHWWtagger_events_number

[9181.0,
 9181.0,
 9181.0,
 9181.0,
 9181.0,
 9181.0,
 9181.0,
 9181.0,
 9181.0,
 9181.0,
 9181.0]

In [114]:
for i in range(len(cmd_events)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88


In [117]:
cmd_events

'grep "load_events" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'

In [113]:
cmd_events = 'grep "load_events" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
events = os.popen(cmd_events).readlines()
events[i]

IndexError: list index out of range

In [106]:
cmd_HHWWtagger_events = 'grep "my_HHWW_Preselection : event set" /eos/user/z/zhenxuan/hhwwgg/GluGluToRadionToHHTo2G4Q_M300_2017/*/*.log'
HHWWtagger_events = os.popen(cmd_HHWWtagger_events).readlines()
float(HHWWtagger_events[0].split('(')[0].split()[-1])



5030.0

In [ ]:
ak.num(events_nano.Photon.pt)

In [ ]:
padded_objects = ak.pad_none(events['Photon'],4, clip=True)
print(padded_objects)
print(padded_objects["pt"])
data = ak.fill_none(padded_objects["pt"][:,1], -999),
print(data)


In [ ]:
events_nano["Events"].keys()

In [ ]:
events.fields

In [ ]:
ak.num(events.LeadPhoton)

In [ ]:
# test
list_W1 = [
jet1_invM+jet2_invM,jet1_invM+jet3_invM,jet1_invM+jet4_invM,jet1_invM+jet5_invM,jet1_invM+jet6_invM,
jet2_invM+jet3_invM,jet2_invM+jet4_invM,jet2_invM+jet5_invM,jet2_invM+jet6_invM,
jet3_invM+jet4_invM,jet3_invM+jet5_invM,jet3_invM+jet6_invM,
jet4_invM+jet5_invM,jet4_invM+jet6_invM,
jet5_invM+jet6_invM]

### TODO: Find the W mass but now only have closest to 80, need to add two others two close to 125

In [ ]:
# all 6 jet with 4D lorentz Vector
jet1_invM = ak.zip(
    {
    "pt": events.jet_1_pt[:,0],
    "phi": events.jet_1_phi[:,0],
    "eta": events.jet_1_eta[:,0],
    "mass": events.jet_1_mass[:,0]
    }, with_name="Momentum4D")
jet2_invM = ak.zip(
    {
    "pt": events.jet_2_pt[:,0],
    "phi": events.jet_2_phi[:,0],
    "eta": events.jet_2_eta[:,0],
    "mass": events.jet_2_mass[:,0]
    }, with_name="Momentum4D")
jet3_invM = ak.zip(
    {
    "pt": events.jet_3_pt[:,0],
    "phi": events.jet_3_phi[:,0],
    "eta": events.jet_3_eta[:,0],
    "mass": events.jet_3_mass[:,0]
    }, with_name="Momentum4D")
jet4_invM = ak.zip(
    {
    "pt": events.jet_4_pt[:,0],
    "phi": events.jet_4_phi[:,0],
    "eta": events.jet_4_eta[:,0],
    "mass": events.jet_4_mass[:,0]
    }, with_name="Momentum4D")
jet5_invM = ak.zip(
    {
    "pt": events.jet_5_pt[:,0],
    "phi": events.jet_5_phi[:,0],
    "eta": events.jet_5_eta[:,0],
    "mass": events.jet_5_mass[:,0]
    }, with_name="Momentum4D")
jet6_invM = ak.zip(
    {
    "pt": events.jet_6_pt[:,0],
    "phi": events.jet_6_phi[:,0],
    "eta": events.jet_6_eta[:,0],
    "mass": events.jet_6_mass[:,0]
    }, with_name="Momentum4D")

In [ ]:
Higgs = jet1_invM + jet2_invM + jet3_invM + jet4_invM

In [ ]:
test[0][0].rho
with open("file.txt", "w") as output:
    output.write(str(test[0][0].rho))

In [ ]:
CountJet1 = 0
CountJet2 = CountJet1 + 1
nTagJets = len(jet1_invM[0,:])
for CountJet1 in len(nTagJets):
    for CountJet2 in len(nTagJets):

In [ ]:
# test
array_alljets1 = [jet1_invM,jet2_invM,jet3_invM,jet4_invM,jet5_invM,jet6_invM]
array_alljets = []

In [ ]:
for i in range(6):
    for j in range(6):
        array_alljets.append(array_alljets1[i]+array_alljets2[j])

In [ ]:
# test
a = [[1,2,3],[4,5,6]]
print(a[0][1])
b = ak.Array([[1,2,3],[4,5,6]])
print(b[0][1])

In [ ]:
array_alljets = [
    [array_alljets1[0]+array_alljets1[0],array_alljets1[0]+array_alljets1[1],array_alljets1[0]+array_alljets1[2],array_alljets1[0]+array_alljets1[3],array_alljets1[0]+array_alljets1[4],array_alljets1[0]+array_alljets1[5]],
    [array_alljets1[1]+array_alljets1[1],array_alljets1[1]+array_alljets1[1],array_alljets1[1]+array_alljets1[2],array_alljets1[1]+array_alljets1[3],array_alljets1[1]+array_alljets1[4],array_alljets1[1]+array_alljets1[5]],
    [array_alljets1[2]+array_alljets1[2],array_alljets1[2]+array_alljets1[1],array_alljets1[2]+array_alljets1[2],array_alljets1[2]+array_alljets1[3],array_alljets1[2]+array_alljets1[4],array_alljets1[2]+array_alljets1[5]],
    [array_alljets1[3]+array_alljets1[3],array_alljets1[3]+array_alljets1[1],array_alljets1[3]+array_alljets1[2],array_alljets1[3]+array_alljets1[3],array_alljets1[3]+array_alljets1[4],array_alljets1[3]+array_alljets1[5]],
    [array_alljets1[4]+array_alljets1[4],array_alljets1[4]+array_alljets1[1],array_alljets1[4]+array_alljets1[2],array_alljets1[4]+array_alljets1[3],array_alljets1[4]+array_alljets1[4],array_alljets1[4]+array_alljets1[5]],
    [array_alljets1[5]+array_alljets1[5],array_alljets1[5]+array_alljets1[1],array_alljets1[5]+array_alljets1[2],array_alljets1[5]+array_alljets1[3],array_alljets1[5]+array_alljets1[4],array_alljets1[5]+array_alljets1[5]]
]

In [ ]:
for i in range(200):
    test = abs(np.nan_to_num(array_alljets[:,i].tau) - 80)
    print(array_alljets[ak.argsort(test)[0],i].tau)

In [ ]:
# make W1 as awkward array
W1 = ak.Array([jet1_invM+jet2_invM,jet1_invM+jet3_invM,jet1_invM+jet4_invM,jet1_invM+jet5_invM,jet1_invM+jet6_invM,
jet2_invM+jet3_invM,jet2_invM+jet4_invM,jet2_invM+jet5_invM,jet2_invM+jet6_invM,
jet3_invM+jet4_invM,jet3_invM+jet5_invM,jet3_invM+jet6_invM,
jet4_invM+jet5_invM,jet4_invM+jet6_invM,
jet5_invM+jet6_invM])


In [ ]:
W1[0].tau

In [ ]:
for i in range(200):
    test = abs(np.nan_to_num(W1[:,i].tau) - 80)
    print(W1[ak.argsort(test)[0],i].tau)

In [ ]:
test = ak.Array([[80]*15]*23692)

In [ ]:
flatten_W1_mass = ak.flatten(W1.tau)

In [ ]:
print(flatten_W1_mass)

In [ ]:
# test = ak.Array([[1,2,3],[5,6,7]])
print(W1[0,:].tau)

In [ ]:
test = ak.Array([[1,2,3],[4,5,6]])

In [ ]:
tmp_W1_mass = 80
list_tmp_W1 = [jet1_invM+jet2_invM,jet1_invM+jet3_invM,jet1_invM+jet4_invM,jet1_invM+jet5_invM,jet1_invM+jet6_invM,
jet2_invM+jet3_invM,jet2_invM+jet4_invM,jet2_invM+jet5_invM,jet2_invM+jet6_invM,
jet3_invM+jet4_invM,jet3_invM+jet5_invM,jet3_invM+jet6_invM,
jet4_invM+jet5_invM,jet4_invM+jet6_invM,
jet5_invM+jet6_invM]
list_W1 = []
list_W1_pt = []
list_W1_eta = []
list_W1_phi = []
list_W1_mass = []
# events i , W1 index j
for i in range(len(jet1_invM[:])):
# for i in range(1):
    min_deltaMass = 10000
    for j in range(len(list_tmp_W1)):
        # print("test")
        if(abs(list_tmp_W1[j].mass[i]-tmp_W1_mass) < min_deltaMass):
            min_deltaMass = abs(list_tmp_W1[j].mass[i]-tmp_W1_mass)
            index = j
        
    list_W1.append(list_tmp_W1[index][i])
    list_W1_pt.append(list_tmp_W1[index][i].pt)
    list_W1_eta.append(list_tmp_W1[index][i].eta)
    list_W1_phi.append(list_tmp_W1[index][i].phi)
    list_W1_mass.append(list_tmp_W1[index][i].mass)






In [ ]:
len(jet1_invM[:])

In [ ]:
list_W1__mass = []
for i in range(len(list_W1)):
    list_W1_mass.append(list_W1[i].mass)

In [ ]:
events.fatjet_1_pt[:,0]

In [ ]:
plt.hist(all_list[0])

In [ ]:
minx = 0
maxx = 500
array_jet5 = events.SelectedJet_pt[:,4:5]
array_jet6 = events.SelectedJet_pt[:,5:6]
array_jet7 = events.SelectedJet_pt[:,6:7]
array_jet8 = events.SelectedJet_pt[:,7:8]
pick_color = list(mcolors.TABLEAU_COLORS)
# all_list = [events.SelectedJet_pt[:,0],events.SelectedJet_pt[:,1],events.SelectedJet_pt[:,2],events.SelectedJet_pt[:,3]]
# all_list = [events.fatjet_1_pt[:,0][events.fatjet_1_pt[:,0]>0],events.fatjet_2_pt[:,0][events.fatjet_2_pt[:,0]>0],events.fatjet_3_pt[:,0][events.fatjet_3_pt[:,0]>0]]
all_list = [Higgs.mass]
# all_list = [events.fatjet_1_pt[:,0][events.fatjet_1_pt[:,0]>0]]
# array_jet5[ak.num(array_jet5)>0][:,0],array_jet6[ak.num(array_jet6)>0][:,0],array_jet7[ak.num(array_jet7)>0][:,0],array_jet8[ak.num(array_jet8)>0][:,0]]
# all_list = [reduce(add, events.SelectedJet_pt.to_list())]
# name_list = ["sig_pt1","sig_pt2","sig_pt3","sig_pt4","bkg_pt1","bkg_pt2","bkg_pt3","bkg_pt4"]
name_list = ["H_mass"]
# all_list = all_list_sig # for signal
# all_lsit = all_list_bkg # for bkg
hep.cms.label(rlabel='')
for i in range(0,len(all_list)):
    plt.hist(all_list[i],
             bins=100,
                range=(minx,maxx),
                label=name_list[i],
                fill=False,
                linewidth=2,
                edgecolor=pick_color[i],
                histtype='step'
                )
plt.xlabel("mass")
# plt.xlabel(r"$pT(GeV)$")
plt.ylabel("number of events")
# plt.title("Order with Pt")
# plt.title("with pt>15GeV cut")
# plt.title("Higgs mass")
plt.legend(loc=1,fontsize=20)
# plt.axvline(x = 125, color = 'r',label="M = 125",linestyle ="--")
# plt.legend(loc=1,fontsize=10)

### the way to convert unregular ak.array to 1D list

In [ ]:
a = reduce(add, events.Diphoton_mass.to_list())

### plots the hist

In [ ]:
array = events.SelectedJet_pt[:,4:5]
array[ak.num(array)>0][:,0]

In [ ]:
minx = 0
maxx = 100
array_jet5 = events.SelectedJet_pt[:,4:5]
array_jet6 = events.SelectedJet_pt[:,5:6]
array_jet7 = events.SelectedJet_pt[:,6:7]
array_jet8 = events.SelectedJet_pt[:,7:8]
pick_color = list(mcolors.TABLEAU_COLORS)
# all_list = [events.SelectedJet_pt[:,0],events.SelectedJet_pt[:,1],events.SelectedJet_pt[:,2],events.SelectedJet_pt[:,3],
# array_jet5[ak.num(array_jet5)>0][:,0],array_jet6[ak.num(array_jet6)>0][:,0],array_jet7[ak.num(array_jet7)>0][:,0],array_jet8[ak.num(array_jet8)>0][:,0]]
all_list = [events.SelectedJet_pt[:,0],events.SelectedJet_pt[:,1],events.SelectedJet_pt[:,2],events.SelectedJet_pt[:,3]]
# all_list = [reduce(add, events.SelectedJet_pt.to_list())]
name_list = ["sig_pt1","sig_pt2","sig_pt3","sig_pt4","bkg_pt1","bkg_pt2","bkg_pt3","bkg_pt4"]
# all_list = all_list_sig # for signal
# all_lsit = all_list_bkg # for bkg
hep.cms.label(rlabel='')
for i in range(0,len(all_list)):
    plt.hist(all_list[i],
             bins=100,
                range=(minx,maxx),
                label=name_list[i],
                fill=False,
                linewidth=2,
                edgecolor=pick_color[i],
                histtype='step'
                )
plt.xlabel("pt")
# plt.xlabel(r"$pT(GeV)$")
plt.ylabel("number of events")
# plt.title("Order with Pt")
# plt.title("with pt>15GeV cut")
# plt.title("Higgs mass")
plt.legend(loc=1,fontsize=20)
# plt.axvline(x = 125, color = 'r',label="M = 125",linestyle ="--")

In [ ]:
jets = ak.Array([[1,2,3],[4,5,6,7]])

In [ ]:
a = ak.pad_none(jets,5,clip=True)

In [ ]:
print(a)